# ABCD - It is a technique used for detecting the probability of a lesion being a melanoma.
### A - Asymmetry
The asymmetry of the lesion. The more asymmetrical the more suspectful it is of being melanoma.
### B - Border
### C - Colour
### D - Dimension

In [27]:
import cv2 #Image Library
import numpy as np #Numpy library

imname = "Images/FirstImages/ISIC_0024792.png" #Name of the image to analyze
image = cv2.imread(imname)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #Same image but in grayscale, this is done for calculating the threshold
_,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV) #Get the threshold, separate the melanoma from healthy skin
#It is done to binary, so all values are sent to either extreme, 0 or 1
contours, hier = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)#Get the countours, returned to contrs
cv2.drawContours(image, contours, -1, (0,255,0), 3) #Draw the countours onto colour image

cv2.imshow('Image1',image)
cv2.waitKey(0)
cv2.destroyAllWindows()